In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.llms import GPT4All
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader

from langchain_classic.chains.history_aware_retriever import create_history_aware_retriever
from langchain_classic.chains.retrieval import create_retrieval_chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.summarize import load_summarize_chain

In [37]:
          ##loading docs##


# Path to your PDF document
pdf_path =r"C:\Users\Mekzedeck\Desktop\ChatBot\nhif.pdf"

# Initialize the PDF loader
loader = PyPDFLoader(pdf_path)

# Load the PDF content into a list of documents
documents = loader.load()

# loader=WebBaseLoader("http://selfservice.nhif.or.tz/home/")
# documents = loader.load()

# Each document represents a section of the PDF
for i, doc in enumerate(documents):
     print(f"Document {i+1} content:\n", doc.page_content)
    
#Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

#Initialize an embedding model (using GPT4All in this case)
embeddings = GPT4AllEmbeddings()


# Use Chroma for storing and managing embeddings
vector_store = Chroma.from_documents(texts, embeddings)

retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={
                              'score_threshold': 0.1, 'k': 10})


#define llm
llm = GPT4All(
    model=r"D:\GPT4all\resources\nomic-embed-text-v1.5.f16.gguf",
    backend="llama",
    verbose=True
)




Document 1 content:
 i 
 
NATIONAL HEALTH INSURANCE FUND 
 
 
 
 
 
 
 
ANNUAL REPORT FOR THE FINANCIAL YEAR 2022/23 
 
 
 
 
 
 
 
 
 
 
NHIF-Access to Health Care for All
Document 2 content:
 ii 
 
CORPORATE INFORMATION 
 
National Health Insurance Fund (NHIF), 
Head Office, NHIF House, 2 Ukaguzi Road, 
41101 Tambukareli, 
P.O. Box 1437, 
DODOMA. 
E-mail: info@nhif.or.tz 
PARENT MINISTRY 
Ministry of Health, 
Afya Street – Mtumba, 
P.O. Box 743, 
40478 DODOMA 
REGULATOR 
Bank of Tanzania (BOT)  
16 Jakaya Kikwete Road,  
P.O. Box 2303,  
40184 DODOMA 
AUDITORS 
The Controller and Auditor General (CAG),  
National Audit Office of Tanzania,  
4 Ukaguzi Road – Tambukareli, 
P.O. Box 950, 
41104 DODOMA 
BANKERS 
CRDB Bank PLC,                      
P.O. Box 268, 
DAR ES SALAAM 
 
National Bank of Commerce, 
P.O. Box 1863, 
DAR ES SALAAM 
NMB Bank PLC, 
P.O. Box 9213, 
DAR ES SALAAM 
 
Tanzania Commercial Bank Ltd, 
P.O. Box 93000, 
DAR ES SALAAM 
 
Azania Bank Ltd, 
P.O. Box 9271, 
DAR E

In [38]:
### construct question ###
contextextualize_q_system_prompt=(
   """As an AI assistant you provide answers based on the given context, ensuring accuracy and brifness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to loaded documents
        -Only use examples if explicitly requested as reffered to the loaded documents
        -Do not introduce examples outside of the context
        -Never answer if context is Absent 
        """
)
contextextualize_q_system_prompt=ChatPromptTemplate.from_messages(
    [
        ("system", contextextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)
history_aware_retriever=create_history_aware_retriever(
    llm,retriever,contextextualize_q_system_prompt
)

### Answer question ###
system_prompt=(
    """   You are an assistant for queation-answering tasks.

    use the following pieces of retrieved context to answer the question.
     
        You always follow these guidelines:

    -If the answer isn't available within the context, state that fact
    -Otherwise, answer to your best capability, refering to source of documents provided
    -Only use examples if explicitly requested in response to the loaded document
    -Do not introduce examples outside of the context
    -Never answer if context is absent

       =
        
    """
)
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

question_answer_chain = load_summarize_chain(llm, chain_type="stuff")

rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

## Statefully manage chat history
store={}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [39]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# Simple prompt
prompt = PromptTemplate.from_template("Answer: {question}")

# Chat prompt (most common)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])


In [40]:
def query_rag(question: str, session_id: str = "abc123"):
    docs = retriever.invoke(question)
    return conversational_rag_chain.invoke(
        {"input": question, "input_documents": docs},
        config={"configurable": {"session_id": session_id}}
    )


In [41]:
def main():
    session_id = "abc123"

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Conversation ended.")
            break

        result = conversational_rag_chain.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )

        response = result.get("answer", result)
        print(f"Assistant: {response}")

if __name__ == "__main__":
    main()


ValueError: Missing some input keys: {'input_documents'}

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message,AIMessage):
        prefix="AI"
    else:
        prefix="User"
    
    print(f"{prefix}:{message.content}\n")

KeyError: 'abc123'